In [35]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import pandas as pd
import time

In [36]:
# Definimos variables globales:
SCROLL_PAUSE_TIME = 0.5
DRIVER_DIR = "C:\\chromedriver_win32\\chromedriver.exe"
OUTPUT_DIR = "C:\\Users\\Matias\\Documents\\Matias\\Repositorios\\de_ruedas_ws\\outputs"

In [37]:
# Especificamos mapeos
map_order = {
    'Destacados': 1,
    'Más Nuevos': 2,
    'Más Antiguos': 3,
    'Más Caros': 4,
    'Más Baratos': 5,
    'Últimos Publicados': 6,
}

def get_id_precio(px):
    return int(px/100000)

In [210]:
# definimos parámetros de búsqueda
order = 'Últimos Publicados'
precio_min = 0 
precio_max = 5_000_000
kilometraje_max = 150000
anio_min = 2013
anio_max = datetime.now().year
brands = [
        #   'Alfa Romeo',
        #   'Audi',
        #   'BMW',
        #   'Chery',
        'Chevrolet',
        #   'Citroen',
        #   'DS',
        # 'Fiat',
        #   'Ford',
        #   'Geely',
        #   'Haval',
        #   'Honda',
        #   'Hyundai',
        #   'Jeep',
        #   'Kia',
        #   'Land Rover',
        #   'Mercedes Benz',
        #   'Mini Cooper',
        #   'Nissan',
        #   'Peugeot',
        #   'Renault',
        #   'Toyota',
        #   'Volkswagen',
          ]

### Función para cargar todos los autos de una marca

In [43]:
def get_browse_charged(brand):
    link = f"https://www.deruedas.com.ar/bus.asp?segmento=0&anio_1={anio_min}&anio_2={anio_max}&kilometraje={kilometraje_max}&precio_1={get_id_precio(precio_min)}&precio_2={get_id_precio(precio_max)}&order={map_order[order]}&marca={brand}"
    browser = webdriver.Chrome(DRIVER_DIR)
    browser.get(link)

    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    return browser

### Cargar la página por marca, ingresar a cada auto y guardar la data.

In [ ]:
pd_datos_autos = pd.DataFrame()
for brand in brands:
    browser = get_browse_charged(brand)

    soup = BeautifulSoup(browser.page_source,"html.parser",multi_valued_attributes=None)
    soup_autos = soup.findAll("div", {"id": lambda value: value and value.startswith("car_")})

    lst_link_autos = []
    for auto in soup_autos:
        lst_link_autos.append(f'https://www.deruedas.com.ar/{auto.find("a").get("href")}')

    lst_dict_datos_auto = []
    for link_auto in lst_link_autos:
        dict_datos_auto = {}
        dict_datos_auto['day'] = str(datetime.now().date())
        dict_datos_auto['marca'] = brand
        dict_datos_auto['link'] = link_auto


        browser.get(link_auto)

        # Cuadro superior (info principal)
        soup = BeautifulSoup(browser.page_source,"html.parser",multi_valued_attributes=None)
        soup_info_principal = soup.find("div", {"id": 'divInfoPrincipal'})

        dict_datos_auto['nombre'] = soup_info_principal.find('span', {'class': 'titulo resaltar'}).find('h1').text
        dict_datos_auto['version'] = soup_info_principal.find('span', {'class': 'titulo resaltar'}).find('span').text

        # información de contacto
        soup_info_contacto = soup.find("div", {"id": 'divContactoVehiculo'})

        try:
            dict_datos_auto['link_contacto'] = soup_info_contacto.find('tbody').findAll('a')[1].get('href')
        except:
            dict_datos_auto['link_contacto'] = None
        try:
            dict_datos_auto['nombre_contacto'] = soup_info_contacto.find('tbody').findAll('a')[1].text
        except:
            try:
                dict_datos_auto['nombre_contacto'] = soup_info_contacto.find('tbody').find('span').text
            except:
                dict_datos_auto['nombre_contacto'] = soup_info_contacto.find('tbody').find('b').text
        dict_datos_auto['ubicacion'] = soup_info_contacto.find('h2').text

        # Detalles en recuadros
        soup_info_datos = soup.find('div', {'id': 'divFichaVehiculo'}).findAll("div", {"class": 'recuadro'})
        lst_datos = soup_info_datos[0].findAll('b')

        lst_attrs = [
            'segmento',
            'ano',
            'combustible',
            'kilometraje',
            'precio',
            'permuta_por_menor',
            'permuta_por_mayor',
            'ultima_actualizacion',
            'provincia',
            'visitas'
            ]


        for i, dato in enumerate(lst_datos):
            dict_datos_auto[lst_attrs[i]] = dato.text

        dict_datos_auto['equipamiento'] = soup_info_datos[1].find('div', {'class':'texto'}).text
        dict_datos_auto['comentarios_vendedor'] = soup_info_datos[2].find('div', {'class':'texto'}).text.replace('\n','').replace('\t','')

        lst_dict_datos_auto.append(dict_datos_auto)


        # print(dict_datos_auto)
    pd_datos_autos = pd.concat([pd_datos_autos, pd.DataFrame(lst_dict_datos_auto)])

In [248]:
set_equip = set()
for equip in pd_datos_autos['equipamiento'].str.split(','):
    equip = [eq.strip() for eq in equip]
    set_equip.update(set(equip))


lst_equip = list(set_equip)
for equip in list(set_equip):
    pd_datos_autos[equip] = [1 if equip in lst_eq else 0 for lst_eq in pd_datos_autos['equipamiento']]

def elimina_tildes(txt):
    return txt.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')


pd_datos_autos.columns = [elimina_tildes(eq.lower().replace(' / ','/').replace(' ','_')) for eq in pd_datos_autos.columns]

In [284]:
pd_datos_autos.to_csv('chevrolet_11062022.csv')